## Data Collection Task

### Mô tả công việc:
1. Trang web cào dữ liệu:\
   a. [kokotaru](https://kokotaru.com/) - Tổng: ~ 500 Bài viết.\
   b. [kitchenart](https://cook.kitchenart.vn/cong-thuc-nau-an/) - Tổng: ~ 800 bài viết.
2. Dữ liệu cần cào:

   | Tên món ăn | Các thành phần chính |
   | -----------| ---------------------|

### Cài đặt thư viện

In [ ]:
# !pip install bs4
# !pip install selenium
# !pip install numpy
# !pip install pandas
# !pip install requests
# !pip install tqdm

### Import các thư viện và các hàm dùng chung

In [1]:
from Libraries_Used import *
from Shared_Functions import *

### **Nhận xét:**
Vì hai trang web này có cách hoạt động và tương tác khác nhau cho nên sẽ có 2 phần thu thập thông tin riêng biệt.
1. **Kokotaru:** Tương tác cuộn chuột để load thêm các bài viết.
2. **Kitchenart:** Phân ra các trang riêng biệt, mỗi trang gồm 20 bài viết.

### KOKOTARU WEBSITE ARTICLE URLS PARSING

#### Tương tác:
* Load page theo thao tác cuộn chuột của người dùng.
#### Mô tả:
* Tất cả bài viết được tập hợp chung ở một link homepage của kokotaru.

### **Bước 1: Lấy ra nội dung của trang web bằng thư viện Selenium.**

* Định dạng đường dẫn cơ bản

In [ ]:
KOKOTARU_BASE_URL = 'https://kokotaru.com/'

**Bước 1.1:** Tạo selenium chrome browser.\
**Bước 1.2:** Cho trang load hết tất cả nội dung.

In [ ]:
def kokotaru_page_loader(BASE_URL: str = KOKOTARU_BASE_URL) -> str:
    
    driver = webdriver.Chrome()

    driver.get(BASE_URL)

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    progress_bar = tqdm(desc="Scrolling", ncols=100, leave=True, unit="lần")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        
        progress_bar.update(1)
        
        if new_height == last_height:
            break
        
        last_height = new_height

    html_content = driver.page_source

    driver.quit()
    
    progress_bar.close() 
    print("Parsing completed.")
    
    return html_content

In [ ]:
kokotaru_html = kokotaru_page_loader(KOKOTARU_BASE_URL)

**Bước 1.3:** Lấy nội dung cho vào `kokotaru_html_content.html`

In [ ]:
write_to_file("Assert/kokotaru_html_content.html",kokotaru_html, 'w_b_str')

#### **Bước 2: Lấy ra các link bài viết có trong homepage**

In [ ]:
kokotaru_content = read_from_file("Assert/kokotaru_html_content.html", 'r_b_str')

In [ ]:
def get_kokotaru_articles_urls(html_content) -> list:
    urls = []
    soup = BeautifulSoup(html_content, 'html.parser')
    article_headers = soup.find_all("article", { "class" : "entry-preview" })
    for header in article_headers:
        found = header.find('a', class_='cs-overlay-link')
        url = found['href'] if found else None
        if url is not None:
            urls.append(url)
    return urls

kokotaru_article_urls = get_kokotaru_articles_urls(kokotaru_content)


Ghi lại các link urls lấy được vào trong `kokotaru_aricle_urls.txt`

In [ ]:
write_to_file('Assert/kokotaru_aricle_urls.txt', kokotaru_article_urls, 'w_b_element')

### KITCHENART WEBSITE PARSING

#### Tương tác:
* Phân trang tổng hợp.
#### Mô tả:
* Mỗi trang gồm 20 bài viết.

### **Bước 1: Lấy ra các urls dẫn đến các bài viết có trong trang hiện tại**

In [2]:
def kitchenart_article_urls_onepage(page_url: str) -> list:
    u_list = []

    response = requests.get(page_url)
    
    if response.status_code == 200:
        try:
            soup = BeautifulSoup(response.content, "html.parser")
            
            articles_list = soup.find_all("a", {"class": "recipe-card__title-link"})
            
            for article in articles_list:
                link = article.get('href') if article.get('href') else None
                if link is not None:
                    u_list.append(link)
        
        except Exception as err:
            print(f'Requests error: {err}')
    else:
        print(f"Failed to access {page_url}. Status code: {response.status_code}")
    
    
    return u_list

### **Bước 2: Lấy ra url dẫn đến trang tiếp theo**

In [3]:
def kitchenart_find_next_page_url(current_page: str) -> str:
    
    next_page = ""
    
    response = requests.get(current_page)
    
    if response.status_code == 200:
        try:
            soup = BeautifulSoup(response.content,"html.parser")

            link_found = soup.find("a", {"class" : "next page-numbers"}) if soup.find("a", {"class" : "next page-numbers"}) else None
                
            if link_found is not None:
                
                next_page = link_found.get("href")
            
        except Exception as err:
            print(f'Requests error: {err}')
    else:
        print(f"Failed to access {current_page}. Status code: {response.status_code}")
        
    return next_page

### **Bước 3: Lấy ra tất cả các urls bài viết trong tất cả các trang hiện có**

In [4]:
KITCHENART_BASE_URL = 'https://cook.kitchenart.vn/cong-thuc-nau-an/'

In [5]:
def find_all_article_urls_in_all_pages(first_url: str) -> list:
    u_list = []
    current_page = first_url
    page_counter = 1
    progress_bar = tqdm(desc="Page Loading", unit="page", initial=1)
    
    while current_page:
        tmp_url_list = kitchenart_article_urls_onepage(current_page)
        u_list.extend(tmp_url_list)
        
        next_page = kitchenart_find_next_page_url(current_page)
        
        if next_page == '':
            break
        
        current_page = next_page
        page_counter += 1
        progress_bar.update(1)
    
    progress_bar.close()
    return u_list

### **Bước 4: Chạy chương trình để lấy ra tất cả các article urls**

In [6]:
kitchenart_article_urls = find_all_article_urls_in_all_pages(KITCHENART_BASE_URL)

Page Loading: 28page [00:19,  1.52page/s]

Failed to access https://cook.kitchenart.vn/cong-thuc-nau-an/page/28/. Status code: 520


Page Loading: 28page [00:19,  1.37page/s]

Failed to access https://cook.kitchenart.vn/cong-thuc-nau-an/page/28/. Status code: 403


**Bước 4.1:** Ghi các urls này vào file `kitchenart_article_urls.txt`

In [7]:
write_to_file('Assert/kitchenart_aricle_urls.txt', kitchenart_article_urls, 'w_b_element')